In [279]:
import pandas as pd
import numpy as np

In [280]:
wncaa=pd.read_csv("wnba.csv", index_col=0)
wncaa=wncaa.reset_index(drop=True)

In [281]:
wncaa.head()

,TeamID,Season,games,totalMargin,Wins,Losses,Points,FG,FGA,FG3,FGA3,FT,FTA,OR,DR,Ast,To,Stl,Blk,Pf
0,3102.0,2010.0,28,-19.964286,1,27,51.821429,19.142857,53.142857,4.571429,16.071429,8.964286,13.214286,12.464286,19.535714,10.250000,17.714286,6.035714,0.642857,14.964286
1,3103.0,2010.0,30,2.600000,17,13,62.233333,22.233333,55.266667,3.933333,12.466667,13.833333,19.600000,14.200000,23.566667,13.433333,18.266667,8.033333,2.400000,17.533333
2,3104.0,2010.0,29,-2.862069,11,18,64.137931,24.724138,62.103448,4.344828,14.724138,10.344828,16.793103,14.482759,25.931034,13.103448,19.172414,7.379310,2.931034,20.413793
3,3105.0,2010.0,27,-3.962963,14,13,60.777778,20.370370,50.851852,3.037037,9.888889,17.000000,24.333333,13.666667,23.370370,9.740741,25.037037,8.777778,5.148148,19.148148
4,3106.0,2010.0,29,-2.275862,12,17,55.137931,18.448276,53.000000,2.689655,9.586207,15.551724,24.000000,15.000000,24.758621,11.482759,20.103448,8.034483,4.482759,22.068966


In [282]:
tourn = pd.read_csv("google-cloud-ncaa-march-madness-2020-division-1-womens-tournament/WDataFiles_Stage1/WNCAATourneyDetailedResults.csv")

In [283]:
sample=pd.read_csv("google-cloud-ncaa-march-madness-2020-division-1-womens-tournament/WSampleSubmissionStage1_2020.csv")

In [284]:
id = sample["ID"].str.split("_", n = 2, expand = True)

In [285]:
sample["Season"] = id[0]
sample ["Team1"] = id[1]
sample ["Team2"] = id[2]


In [286]:
tourn2 = tourn[["Season", "WTeamID", "LTeamID", "WScore", "LScore"]]
tourn2["Result"]=1

/home/sam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [287]:
cols=['games', 'totalMargin', 'Wins', 'Losses', 'Points',
       'FG', 'FGA', 'FG3', 'FGA3', 'FT', 'FTA', 'OR', 'DR', 'Ast', 'To', 'Stl',
       'Blk', 'Pf']
team1=['games_x', 'totalMargin_x', 'Wins_x', 'Losses_x',
       'Points_x', 'FG_x', 'FGA_x', 'FG3_x', 'FGA3_x', 'FT_x', 'FTA_x', 'OR_x',
       'DR_x', 'Ast_x', 'To_x', 'Stl_x', 'Blk_x', 'Pf_x']

team2=[
       'games_y', 'totalMargin_y', 'Wins_y', 'Losses_y', 'Points_y', 'FG_y',
       'FGA_y', 'FG3_y', 'FGA3_y', 'FT_y', 'FTA_y', 'OR_y', 'DR_y', 'Ast_y',
       'To_y', 'Stl_y', 'Blk_y', 'Pf_y']

In [288]:
import random
random.seed(42)
rand = random.sample(range(0,63), 30)
seasons = tourn2.Season.unique()

def cleanTourn():
    clean = pd.DataFrame()
    for season in seasons:
        to = tourn2[tourn2["Season"]==season]
        to=to.reset_index()
        for num in rand:
            to.loc[num,["WTeamID", "LTeamID"]] = to.loc[num, ["LTeamID", "WTeamID"]].values
            to.loc[num,["WScore", "LScore"]] = to.loc[num, ["LScore", "WScore"]].values
            to.loc[num,["Result"]] = 0
        clean=pd.concat([clean,to])
    clean=clean.rename(columns={"WTeamID": "Team1", "LTeamID": "Team2", "WScore": "Team1Score", "LScore": "Team2Score"})
    clean=clean[["Season", "Team1", "Team2", "Team1Score", "Team2Score", "Result"]]
    return clean
train=cleanTourn()


In [289]:
train.head()

,Season,Team1,Team2,Team1Score,Team2Score,Result
0,2010,3124,3201,69,55,1
1,2010,3395,3173,66,67,0
2,2010,3214,3181,37,72,0
3,2010,3199,3256,75,61,1
4,2010,3207,3265,62,42,1


In [291]:
train=pd.merge(train, wncaa, left_on=["Season", "Team1"], right_on=["Season", "TeamID"])
train=pd.merge(train, wncaa, left_on=["Season", "Team2"], right_on=["Season", "TeamID"])

(630, 25)
(630, 44)


In [292]:
sample[["Season", "Team1", "Team2"]] = sample[["Season", "Team1", "Team2"]].apply(pd.to_numeric)

In [294]:
test = pd.merge(sample, wncaa, left_on=["Season", "Team1"], right_on = ["Season", "TeamID"])

print(test.shape)
test = pd.merge(test, wncaa, left_on=["Season", "Team2"], right_on = ["Season", "TeamID"])
def cleanTest():
    featureSpace = ["Season", "Team1", "Team2"]
    for i in range(len(team1)):
        colname = "diff_" + cols[i]
        test[colname]=test[team1[i]] - test[team2[i]]
        featureSpace.append(colname)
    return test[featureSpace]

(10080, 24)


In [295]:
def cleanTrain():
    featureSpace = ["Season", "Team1", "Team2", "Result"]
    for i in range(len(team1)):
        colname = "diff_" + cols[i]
        train[colname]=train[team1[i]] - train[team2[i]]
        featureSpace.append(colname)
    return train[featureSpace]

In [296]:
cleanTrain=cleanTrain()

In [297]:
cleanTest=cleanTest()

In [298]:
result = train.Result

In [310]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
features=['diff_totalMargin',
       'diff_Wins', 'diff_Losses', 'diff_Points', 'diff_FG', 'diff_FGA',
       'diff_FG3', 'diff_FGA3', 'diff_FT', 'diff_FTA', 'diff_OR', 'diff_DR',
       'diff_Ast', 'diff_To', 'diff_Stl', 'diff_Blk', 'diff_Pf']


In [311]:
log.fit(cleanTrain[cleanTrain["Season"]<2015][features], cleanTrain[cleanTrain["Season"]<2015]["Result"])

/home/sam/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [312]:
predictions = log.predict_proba(cleanTest[features])

In [ ]:
result

In [313]:
sample2=pd.read_csv("google-cloud-ncaa-march-madness-2020-division-1-womens-tournament/WSampleSubmissionStage1_2020.csv")

In [305]:
sample2.Pred =predictions

In [308]:
sample2.to_csv("wnba_log.csv", index=False)